# Forest Cover Type Prediction

## Keras - 1 Attempt

In [ ]:

# Reproducibility
import os
import random
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Set seeds
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Data loading
train_files = ['train.csv.zip', 'train.csv.zip']
train_dfs = []
for f in train_files:
    try:
        train_dfs.append(pd.read_csv(f))
    except Exception:
        pass

df_test = None
for f in ['test.csv.zip', 'test.csv.zip']:
    try:
        df_test = pd.read_csv(f)
        break
    except Exception:
        pass

df = pd.concat(train_dfs, ignore_index=True)
id_col = 'Id'
target_columns = ['Cover_Type']

# Target encoding
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_enc = le.transform(df[target_columns[0]].astype(str))
classes_ = le.classes_

# Features
X = df.drop(columns=target_columns + [id_col], errors='ignore')

if df_test is None:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
    )
    X_train = X_train.drop(columns=[id_col], errors='ignore')
    X_val = X_val.drop(columns=[id_col], errors='ignore')
    test_ids = None
else:
    X_train = X
    y_train = y_enc
    test_ids = df_test[id_col]
    X_val = df_test.drop(columns=target_columns + [id_col], errors='ignore')
    y_val = None

# Drop columns with all missing
cols_all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train = X_train.drop(columns=cols_all_missing)
X_val = X_val.drop(columns=cols_all_missing)

# Detect columns
d_numeric = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols = [c for c in cat_cols if X_train[c].nunique() <= 50]

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', cat_transformer, cat_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture
n_samples, n_features = X_train_proc.shape
layers = []
if n_samples < 10000 or n_features < 100:
    units1 = min(n_features * 2, 128)
    units2 = min(n_features, 64)
    dropout_rate = 0.2
    layers += [
        tf.keras.layers.Dense(int(units1), activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(int(units2), activation='relu'),
        tf.keras.layers.Dropout(dropout_rate)
    ]
else:
    units_list = [min(int(n_features * factor), 1024) for factor in (2, 1, 0.5, 0.25)]
    units_list = [u for u in units_list if u >= 16]
    for u in units_list:
        layers.append(tf.keras.layers.Dense(u, activation='relu'))
        layers.append(tf.keras.layers.BatchNormalization())
        layers.append(tf.keras.layers.Dropout(0.3))

num_classes = len(classes_)
top_k = min(num_classes, 5)
output_activation = 'softmax'
loss = 'sparse_categorical_crossentropy'
batch_size = 64
metrics = [
    'accuracy',
    SparseTopKCategoricalAccuracy(k=top_k, name=f'top_{top_k}_accuracy')
]

inputs = tf.keras.Input(shape=(n_features,))
x = inputs
for layer in layers:
    x = layer(x)
outputs = tf.keras.layers.Dense(num_classes, activation=output_activation)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss=loss, metrics=metrics)

# Callbacks & Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]
start_time = time.time()
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=batch_size,
        callbacks=callbacks, verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=batch_size,
        callbacks=callbacks, verbose=2
    )
duration = time.time() - start_time

# Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history.get('val_accuracy', [None])[-1],
    'validation_loss': history.history.get('val_loss', [None])[-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_val_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
if final.ndim == 1:
    final = final.reshape(-1, 1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)


2025-07-09 22:38:03.728633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752100683.747262  925990 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752100683.752434  925990 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752100683.766840  925990 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752100683.766856  925990 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752100683.766858  925990 computation_placer.cc:177] computation placer alr

Epoch 1/100


2025-07-09 22:38:08.442417: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


378/378 - 2s - 4ms/step - accuracy: 0.5662 - loss: 1.0788 - top_5_accuracy: 0.9807 - val_accuracy: 0.7068 - val_loss: 0.7557 - val_top_5_accuracy: 0.9998
Epoch 2/100


378/378 - 1s - 3ms/step - accuracy: 0.6587 - loss: 0.8125 - top_5_accuracy: 0.9990 - val_accuracy: 0.7396 - val_loss: 0.6737 - val_top_5_accuracy: 0.9998
Epoch 3/100


...

378/378 - 1s - 2ms/step - accuracy: 0.8532 - loss: 0.3590 - top_5_accuracy: 1.0000 - val_accuracy: 0.8816 - val_loss: 0.2999 - val_top_5_accuracy: 1.0000
Epoch 94/100


378/378 - 1s - 2ms/step - accuracy: 0.8521 - loss: 0.3576 - top_5_accuracy: 1.0000 - val_accuracy: 0.8753 - val_loss: 0.2982 - val_top_5_accuracy: 1.0000
Epoch 95/100
378/378 - 1s - 2ms/step - accuracy: 0.8497 - loss: 0.3602 - top_5_accuracy: 1.0000 - val_accuracy: 0.8763 - val_loss: 0.3007 - val_top_5_accuracy: 1.0000
Epoch 96/100


378/378 - 1s - 2ms/step - accuracy: 0.8534 - loss: 0.3534 - top_5_accuracy: 1.0000 - val_accuracy: 0.8791 - val_loss: 0.2960 - val_top_5_accuracy: 1.0000
Epoch 97/100
378/378 - 1s - 2ms/step - accuracy: 0.8492 - loss: 0.3599 - top_5_accuracy: 1.0000 - val_accuracy: 0.8786 - val_loss: 0.2986 - val_top_5_accuracy: 1.0000
Epoch 98/100


378/378 - 1s - 3ms/step - accuracy: 0.8500 - loss: 0.3605 - top_5_accuracy: 1.0000 - val_accuracy: 0.8776 - val_loss: 0.2932 - val_top_5_accuracy: 1.0000
Epoch 99/100
378/378 - 1s - 2ms/step - accuracy: 0.8552 - loss: 0.3516 - top_5_accuracy: 1.0000 - val_accuracy: 0.8811 - val_loss: 0.2964 - val_top_5_accuracy: 1.0000
Epoch 100/100


378/378 - 1s - 2ms/step - accuracy: 0.8514 - loss: 0.3537 - top_5_accuracy: 1.0000 - val_accuracy: 0.8818 - val_loss: 0.2907 - val_top_5_accuracy: 1.0000
17685/17685 ━━━━━━━━━━━━━━━━━━━━ 10s 589us/step


## Keras Tuner - 1 Attempt

## Attempt 1

In [ ]:
import os
import random
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# Set seeds
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Data loading
train_files = ['train.csv.zip', 'train.csv.zip']
train_dfs = []
for f in train_files:
    try:
        train_dfs.append(pd.read_csv(f))
    except Exception:
        pass

df_test = None
for f in ['test.csv.zip', 'test.csv.zip']:
    try:
        df_test = pd.read_csv(f)
        break
    except Exception:
        pass

df = pd.concat(train_dfs, ignore_index=True)
id_col = 'Id'
target_columns = ['Cover_Type']

# Target encoding
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_enc = le.transform(df[target_columns[0]].astype(str))
classes_ = le.classes_

# Features
X = df.drop(columns=target_columns + [id_col], errors='ignore')

if df_test is None:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
    )
    X_train = X_train.drop(columns=[id_col], errors='ignore')
    X_val = X_val.drop(columns=[id_col], errors='ignore')
    test_ids = None
else:
    X_train = X
    y_train = y_enc
    test_ids = df_test[id_col]
    X_val = df_test.drop(columns=target_columns + [id_col], errors='ignore')
    y_val = None

# Drop columns with all missing
cols_all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train = X_train.drop(columns=cols_all_missing)
X_val = X_val.drop(columns=cols_all_missing)

# Detect columns
d_numeric = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols = [c for c in cat_cols if X_train[c].nunique() <= 50]

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, d_numeric),
    ('cat', cat_transformer, cat_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture
n_samples, n_features = X_train_proc.shape
num_classes = len(classes_)

# Define early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

# Tuner setup
bs = 32  # Example batch size
ep = 20  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()  # Start timing

# Retrain model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
duration = time.time() - start_time  # Calculate duration

# Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history.get('val_accuracy', [None])[-1],
    'validation_loss': history.history.get('val_loss', [None])[-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_val_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
if final.ndim == 1:
    final = final.reshape(-1, 1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 01m 23s]
val_loss: 0.29901883006095886

Best val_loss So Far: 0.24649378657341003
Total elapsed time: 00h 14m 26s
Epoch 1/100


756/756 - 11s - 15ms/step - accuracy: 0.6500 - loss: 0.8312 - val_accuracy: 0.7404 - val_loss: 0.6408
Epoch 2/100


756/756 - 4s - 6ms/step - accuracy: 0.7320 - loss: 0.6378 - val_accuracy: 0.7717 - val_loss: 0.5585
Epoch 3/100


756/756 - 4s - 6ms/step - accuracy: 0.7606 - loss: 0.5761 - val_accuracy: 0.7879 - val_loss: 0.5006
Epoch 4/100


756/756 - 5s - 6ms/step - accuracy: 0.7838 - loss: 0.5225 - val_accuracy: 0.8027 - val_loss: 0.4819
Epoch 5/100


...

756/756 - 4s - 5ms/step - accuracy: 0.9641 - loss: 0.1018 - val_accuracy: 0.9696 - val_loss: 0.0811
Epoch 88/100
756/756 - 4s - 5ms/step - accuracy: 0.9620 - loss: 0.1063 - val_accuracy: 0.9709 - val_loss: 0.0861
Epoch 89/100
756/756 - 4s - 5ms/step - accuracy: 0.9590 - loss: 0.1251 - val_accuracy: 0.9633 - val_loss: 0.1017
Epoch 90/100
756/756 - 4s - 5ms/step - accuracy: 0.9602 - loss: 0.1126 - val_accuracy: 0.9658 - val_loss: 0.1002
Epoch 91/100


756/756 - 4s - 5ms/step - accuracy: 0.9630 - loss: 0.1008 - val_accuracy: 0.9683 - val_loss: 0.0767
Epoch 92/100
756/756 - 4s - 5ms/step - accuracy: 0.9633 - loss: 0.1018 - val_accuracy: 0.9719 - val_loss: 0.0780
Epoch 93/100
756/756 - 4s - 5ms/step - accuracy: 0.9632 - loss: 0.1024 - val_accuracy: 0.9701 - val_loss: 0.0821
Epoch 94/100
756/756 - 4s - 5ms/step - accuracy: 0.9630 - loss: 0.1023 - val_accuracy: 0.9732 - val_loss: 0.0810
Epoch 95/100
756/756 - 4s - 5ms/step - accuracy: 0.9640 - loss: 0.1008 - val_accuracy: 0.9722 - val_loss: 0.0800
Epoch 96/100
756/756 - 4s - 5ms/step - accuracy: 0.9650 - loss: 0.1034 - val_accuracy: 0.9696 - val_loss: 0.0918
Epoch 97/100
756/756 - 4s - 6ms/step - accuracy: 0.9659 - loss: 0.0979 - val_accuracy: 0.9732 - val_loss: 0.0797
Epoch 98/100
756/756 - 4s - 5ms/step - accuracy: 0.9640 - loss: 0.1021 - val_accuracy: 0.9707 - val_loss: 0.0854
Epoch 99/100
756/756 - 4s - 5ms/step - accuracy: 0.9648 - loss: 0.1029 - val_accuracy: 0.9676 - val_loss: 0.0

756/756 - 4s - 5ms/step - accuracy: 0.9642 - loss: 0.1049 - val_accuracy: 0.9740 - val_loss: 0.0700
17663/17685 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-07-17 00:18:55.329320: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 248 bytes spill stores, 248 bytes spill loads

2025-07-17 00:18:55.374797: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 260 bytes spill stores, 260 bytes spill loads

2025-07-17 00:18:55.416495: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 260 bytes spill stores, 260 bytes spill loads

2025-07-17 00:18:55.648591: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 560 bytes spill stores, 560 bytes spill loads

2025-07-17 00:18:55.697107: I external/l

17685/17685 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step


In [ ]:
print(duration)

412.242472
